# Lab 4: Regular Expressions and Text Normalisation

### Learning Outcomes
* Be able to set up a Python and Jupyter notebook environment for text analytics
* Understand how to use regular expressions to preprocess text
* Know how to carry out text normalisation using the NLTK library

### Outline

* Getting started: how to set up your environment, Jupyter notebooks introduction
* Acquiring raw text data
* Regular expressions
* Text normalisation 

### How To Complete This Lab

Read the text and the code then look for 'TODOs' that instruct you to complete some missing code. Aim to work through the lab during the scheduled lab hours. To get help, you can talk to TAs or the lecturer during the labs, post questions to Blackboard (anonymously) or on Teams in the QA channel (with your name), or ask a question in the Wednesday live sessions. 

As you work through the notebooks, please make a note of any code that is unclear to you.

The labs *will not be marked*. However, they will prepare you for the coursework, so try to keep up with the weekly labs and have fun with the exercises! To understand what's going on inside the methods we use here, make sure to watch the lecture videos for the same week.

## Getting Started

### Setting up your environment

We recommend using ```conda``` to create an environment with the correct versions of all the packages you need for these labs. You can install either Anaconda or Miniconda, which will include the ```conda``` program. 

We provide a .yml file that lists all the packages you will need, and the versions that we have tested the labs with. You can use this file to create your environment as follows.

1. Open a terminal. Use the command line to navigate to the directory containing this notebook and the file ```environment.yml```. You can use the command ```cd``` to change directory on the command line.

1. Run the conda program by typing ```conda env create -f environment.yml```, then answer any questions that appear on the command line.

1. Activate the environment by running the command ```conda activate data_analytics```.

1. Relaunch Jupyter: shutdown any running instances, and then type ```jupyter lab``` or ```jupyter notebook``` into your command line, depending on whether you prefer the full Jupyter lab development environment, or the simpler Jupyter notebook.

1. Find this notebook and open it up again.

You should now be ready to go!

The core libraries we will be using in this unit are:

- [Datasets](https://huggingface.co/docs/datasets/), produced by HuggingFace, is a hub for lots of interesting text datasets.
- [NLTK](https://www.nltk.org), a comprehensive NLP library.
- [Scikit-learn](https://scikit-learn.org/stable/user_guide.html), for machine learning and classifier evaluation.
- [Gensim](https://radimrehurek.com/gensim/), for topic modelling.

The libraries above have good documentation, which is available either online (links above) or via Python itself, e.g. `help(numpy.array)` in the Python interpreter. 

### Refreshers for Python and Jupyter

**Skip this part if you're already familiar with Python and Jupyter notebooks.**

This lab assumes you have used Python and Jupyter Notebooks before. 

For an introduction or refresher on Python, see the [Introduction to Python lab](https://github.com/UoB-COMS21202/lab_sheets_public/tree/master/lab_1) or the University of Bristol [Beginning Python](https://milliams.gitlab.io/beginning_python/) course. If you are a beginner with Python, you might also like to look at Chapter 1 in the NLTK book, which also provides a guide for "getting started with Python": https://www.nltk.org/book/ 

You will need to use Python 3, not Python 2, and Python 3.6 or newer are recommended.

The labs will be run on [Jupyter Notebook](http://jupyter.org/), an interactive coding environment embedded in a webpage supporting various programing languages (Python, R, Lua, etc.) through the concept of kernels.  

It allows you to enrich your code with complex comments formatted in Markdown and $\LaTeX$, as well as to place the results of your computation right below your code.

Notebooks are organised in cells which can contain either code (in our case, this will be Python code) or text, which can be easily and nicely formatted using the Markdown notation. 

To edit an already existing cell simply double-click on it. You can use the toolbar to insert new cells, edit and delete them (or use keyboard shortcuts which are very handy to speed up coding). 

Cells can be run, by hitting `shift+enter` when editing a cell or by clicking on the `Run` button at the top. Running a Markdown cell will simply display the formatted text, while running a code cell will execute the commands executed in it. 

**Note**: when you run a code cell, all the created variables, implemented functions and imported libraries will be then available to every other code cell. However, it is commonly assumed that cells will be run sequentially in terms of prerequisites. To reset all variables and functions (for debugging) simply click `Kernel > Restart` from the Jupyter menu.

#### Markdown 

Markdown cells allow you to write fancy and simple comments: all of this is written in Markdown - double click on this cell to see the source. An introduction to Markdown syntax can be found [here](https://daringfireball.net/projects/markdown/syntax).

As Markdown is translated to HTML upon displaying it also allows you to use pure HTML: more details are available [here](https://daringfireball.net/projects/markdown/syntax#html).

Finally, you can also display simple $\LaTeX$ equations in Markdown thanks to `MathJax` support. For inline equations wrap your equation between `$` symbols; for display mode equations use `$$`.

## 1. Acquiring Raw Text Data

Now, let's get some text data! We'll start with the Reuters dataset, which contains financial newswire articles.  Run the code below to download the data from [HuggingFace's datasets hub](https://huggingface.co/datasets/newsgroup):

In [105]:
from datasets import load_dataset

train_dataset = load_dataset(
    "reuters21578", 
    "ModHayes",  # Choose this variant of the dataset
    split="train",
    cache_dir="./data_cache",  # Save the data here 
)

print(f"Training dataset with {len(train_dataset)} instances loaded")

Reusing dataset reuters21578 (./data_cache\reuters21578\ModHayes\1.0.0\98a2ad6a0242627562db83992f9625261854c40a88619322596153a5a16a206c)


Training dataset with 20856 instances loaded


We can access the documents in the dataset like elements in a list. For example, document 3 looks like this:

In [106]:
train_dataset[3]

{'text': 'BankAmerica Corp is not under\npressure to act quickly on its proposed equity offering and\nwould do well to delay it because of the stock\'s recent poor\nperformance, banking analysts said.\n    Some analysts said they have recommended BankAmerica delay\nits up to one-billion-dlr equity offering, which has yet to be\napproved by the Securities and Exchange Commission.\n    BankAmerica stock fell this week, along with other banking\nissues, on the news that Brazil has suspended interest payments\non a large portion of its foreign debt.\n    The stock traded around 12, down 1/8, this afternoon,\nafter falling to 11-1/2 earlier this week on the news.\n    Banking analysts said that with the immediate threat of the\nFirst Interstate Bancorp &lt;I> takeover bid gone, BankAmerica is\nunder no pressure to sell the securities into a market that\nwill be nervous on bank stocks in the near term.\n    BankAmerica filed the offer on January 26. It was seen as\none of the major factors l

# 2. Regular Expressions

## 2.1 Search

Now, let's get to grips with regular expressions. Suppose we want to discover all sentences discussing mergers between companies. A first step would be to find all occurrences of the word 'merger':

In [107]:
import re  # Python regular expressions library

all_matches = []

for article in train_dataset:
    matches = re.findall('merger', article['text'])
    if len(matches) == 0:
        continue
    else:
        all_matches.extend(matches)
    
print(len(all_matches))  # length of the list of matches
for match in set(all_matches):  # Use a set to get a list of the unique matches
    print(match) 

810
merger


This has given us a list of matches in the variable `all_matches`, which all contain the string 'merger', but not the sentences themselves.
This isn't very useful, but we can do better if we define the right regular expression!

Regular expressions define a pattern, rather than a specific string, allowing us to generalise our search and retrieve a many different strings that match the pattern.
In Python, we differentiate a regular expression from a normal string by putting an 'r' character in front of the string.

We can generalise our search by using a _disjunction_, which will match against any one of a set of characters. The disjunction is written inside square brackets. 

Let's try to retrieve instances of the word "merger" followed by any letter. We can write a disjunction that matches any lower case letter as `[a-z]`:

In [108]:
all_matches = []

for article in train_dataset:
    matches = re.findall(r'merger [a-z]', article['text'])
    if len(matches) == 0:
        continue
    else:
        all_matches.extend(matches)
    
print(len(all_matches))  # length of the list of matches
for match in set(all_matches):  # Use a set to get a list of the unique matches
    print(match) 

556
merger u
merger r
merger w
merger t
merger g
merger i
merger h
merger e
merger m
merger c
merger f
merger d
merger a
merger n
merger p
merger b
merger s
merger o
merger l


Our current search only matches a single letter of the word after 'merger'. The length of that following word is variable, so how can we write an expression to match the whole word? 

Here, we can use a special character, '\*', which will match against zero or more repetitions of the preceding regular expression. Let's try it out:

In [109]:
all_matches = []

for article in train_dataset:
    matches = re.findall(r'merger [a-z]* ', article['text'])
    if len(matches) == 0:
        continue
    else:
        all_matches.extend(matches)
    
print(len(all_matches))  # length of the list of matches
for match in set(all_matches):  # Use a set to get a list of the unique matches
    print(match) 

460
merger approved 
merger proxy 
merger involving 
merger talks 
merger in 
merger trasaction 
merger proposal 
merger discussions 
merger to 
merger negotiations 
merger guidelines 
merger agreement 
merger we 
merger called 
merger by 
merger still 
merger plan 
merger within 
merger among 
merger application 
merger last 
merger transaction 
merger this 
merger while 
merger which 
merger is 
merger has 
merger offers 
merger was 
merger calls 
merger since 
merger and 
merger but 
merger following 
merger acquisition 
merger for 
merger should 
merger involves 
merger will 
merger the 
merger because 
merger takes 
merger partner 
merger took 
merger are 
merger between 
merger of 
merger costs 
merger as 
merger each 
merger cannot 
merger notification 
merger expenses 
merger offer 
merger wave 
merger provides 
merger shareholders 
merger specialist 
merger accord 
merger or 
merger into 
merger would 
merger at 
merger completed 
merger appears 
merger on 
merger until 
merge

Look closely at that regular expression in the last cell -- why is there a space after '\*'?

Now, let's try to match the preceding word as well. It would be better to match capital letters as well as lower case, which we can do with the disjunction `[a-zA-Z]`. 

TODO 1: complete the code below to retrieve the words that precede and follow 'merger', including capitalised and lower case words.

In [110]:
all_matches = []

for article in train_dataset:
    
    ### COMPLETE THE CODE HERE
    matches = re.findall(r'[a-zA-Z]* merger [a-zA-Z]*', article['text'])
    
    if len(matches) == 0:
        continue
    else:
        all_matches.extend(matches)
    
print(len(all_matches))  # length of the list of matches
for match in set(all_matches):  # Use a set to get a list of the unique matches
    print(match)

494
the merger facility
their merger completed
from merger of
the merger until
a merger with
this merger has
the merger which
revised merger offers
of merger provides
share merger acquisition
projected merger date
its merger and
proposed merger documents
conversion merger recently
on merger speculation
the merger accord
the merger had
aborted merger agreement
preliminary merger agreement
s merger agreement
a merger offer
the merger each
a merger among
the merger to
rail merger case
step merger agreement
share merger agreement
of merger analysis
formal merger agreement
The merger involves
The merger will
the merger by
and merger talks
the merger 
The merger of
 merger and
a merger agreement
possible merger with
found merger partners
A merger approved
preliminary merger talks
reverse merger last
a merger could
a merger attempt
its merger last
end merger while
the merger it
Rainbow merger and
the merger will
the merger should
the merger has
possible merger but
heightened merger or
the mer

This is starting to look more useful, but we still want to retrieve whole sentences. 

Sentences in English are usually demarcated by punctuation, so let's use the following punctuation marks to identify sentence boundaries: '.', '!', '?'. Those punctuation marks are special characters when used in regular expressions, so to force Python to interpret them literally, we need to put the escape character '\' in front of them. 

Now, we can write a disjunction that matches against the punctuation like this: `[\.\!\?]`.

To retrieve a whole sentence, we would like to match all of the text between two punctuation marks. Besides letters, the text of a sentence contains space characters and new line characters, which appear as '\n'. These are formatting characters that specify a line break. To retrieve a whole sentence, we will need to match against letters, spaces and new line characters.

TODO 2: Retrieve all strings containing 'merger', starting from the preceding punctuation mark, until the following punctuation mark. To do this, modify the disjunction that matches against letters to also match spaces and new line characters.

In [111]:
all_matches = []

for article in train_dataset:
    
    ### COMPLETE THE CODE HERE
    x = "[A-Z][a-z\s]*merger[a-z\s]*[\.\!\?]"
    y = "[^\.\s\!\s\?\s][A-Z][\w\s]*merger[\w\s]*[\.\!\?]"
    matches = re.findall(r'[A-Z][\w\s]*merger[\w\s]*[\.\!\?]', article['text'])
    ########     
    if len(matches) == 0:
        continue
    else:
        all_matches.extend(matches)
print(len(all_matches)) #length of the list of matches
for match in set(all_matches):  # Use a set to get a list of the unique matches
    print(match)
    print(f"\n")  

236
Hughes
merger.


Hoechst and Celanese completed their merger last month.


Allegheny International Inc said its
chairman and chief operating officer will remain in their
positions following the merger of the company with an affiliate
of First Boston Corp.


Caremark Inc and Baxter
Travenol Laboratories Inc jointly announced that Caremark
shareholders will vote on July 31 to approve the previously
announced merger with Baxter.


The companies also said the Securities and Exchange
Commission declared effective as of March 24 the registration
statement covering the securities Chemical will issue to Texas
Bancshares shareholders as part of the merger.


The company said it expects to complete the
merger by June 17.


Quest said it
expects to complete the merger within the next several weeks.


Analysts said the merger continues a trend in the
automobile industry of diversification away from carmaking.


Vernitron said that it currently expects the special
shareholder meeting concerning

So far, we have assumed the sentences consist only of letters, spaces and new lines. Can you think of any characters we have excluded here?

A better way to find all matches would be to use _negation_ to match against any character _except_ the punctuation marks that bound the sentences. A negation will match any character except those specified, which we can write like this: `[^\.\!\?]`, where the '^' indicates the negation.

TODO 3: Modify the expression you used in the previous cell to use the negation `[^\.\!\?]` to try to find all sentences.

In [112]:
all_matches = []

for article in train_dataset:
    
    ### COMPLETE THE CODE HERE
    matches = re.findall(r'(?<=[\.\s|\!\s|\?\s])[A-Z][\w\s]*merger[\w\s]*[.!?]', article['text'])
    ########
    if len(matches) == 0:
        continue
    else:
        all_matches.extend(matches)
print(len(all_matches))  # length of the list of matches
for match in set(all_matches):  # Use a set to get a list of the unique matches
    print(match)
    print(f"\n") 

232
Hoechst and Celanese completed their merger last month.


The companies also said the Securities and Exchange
Commission declared effective as of March 24 the registration
statement covering the securities Chemical will issue to Texas
Bancshares shareholders as part of the merger.


The company said it expects to complete the
merger by June 17.


Quest said it
expects to complete the merger within the next several weeks.


Analysts said the merger continues a trend in the
automobile industry of diversification away from carmaking.


Vernitron said that it currently expects the special
shareholder meeting concerning the merger will be held in
August.


Fifth Third said the dilution in per share
earnings for the merger will be negligible.


The proposed merger that would create a 1.


USG also said shareholders will be asked to approve an
amendment to its restated certificate of incorporation that
includes staggered election of directors and the requirement of
an 80 pct vote on certa


Did your improvement return more matching sentences? 

Are there any problems in our sentence segmentation? 

There are lots more special characters that you can use to form really powerful regular expressions. If you're interested, you can find a complete list [here](https://docs.python.org/3/library/re.html#regular-expression-syntax).

## 2.2 Substitution

We can also use regular expressions during _preprocessing_ to clean up text and prepare it for further analysis. For this, we use regular expression _substitution_, which finds a matching string within a larger piece of text, and replaces it with another string.

Let's use this to clean up the text by removing the line break characters.

In Python, we can use the re.sub() function, which takes three arguments:
1. The expression to match. 
2. The pattern we should replace it with
3. The text to apply the subtitution to. 

To remove the line breaks, the expression in argument 1 can be set to match any non-new line characters with the disjunction `[^\n]*`. 

Here, we will also put the disjunctions inside parentheses, like this: `([^\n]*)`. This specifies that the matching characters are a _group_. This allows the second argument (the replacement pattern) to refer to each _group_ of characters matched by the first argument. 

Look at the code below to see how this is done, then run it to get the result:

In [113]:
print('ORIGINAL TEXT: ')
print(train_dataset[3]['text'])
    
clean_article = re.sub(r'([^\n]*)\n([^\n]*)', r'\1\2', train_dataset[3]['text'])
#clean_article = re.search(r'([^\n]*)\n([^\n]*)', train_dataset[3]['text'])
print('CLEAN TEXT:')
print(clean_article)



ORIGINAL TEXT: 
BankAmerica Corp is not under
pressure to act quickly on its proposed equity offering and
would do well to delay it because of the stock's recent poor
performance, banking analysts said.
    Some analysts said they have recommended BankAmerica delay
its up to one-billion-dlr equity offering, which has yet to be
approved by the Securities and Exchange Commission.
    BankAmerica stock fell this week, along with other banking
issues, on the news that Brazil has suspended interest payments
on a large portion of its foreign debt.
    The stock traded around 12, down 1/8, this afternoon,
after falling to 11-1/2 earlier this week on the news.
    Banking analysts said that with the immediate threat of the
First Interstate Bancorp &lt;I> takeover bid gone, BankAmerica is
under no pressure to sell the securities into a market that
will be nervous on bank stocks in the near term.
    BankAmerica filed the offer on January 26. It was seen as
one of the major factors leading the F

In [114]:
### COMPLETE THE CODE HERE
matches = re.findall(r'(?<=[\.\s|\?\s|\!\s])[A-Z][\w\s|\w\n]*[\.\!\?]', train_dataset[3]['text'])
print(len(matches))
print(train_dataset[3]['text'])
#for match in matches:
#       print("<SENTENCE>")
#        print(match)

9
BankAmerica Corp is not under
pressure to act quickly on its proposed equity offering and
would do well to delay it because of the stock's recent poor
performance, banking analysts said.
    Some analysts said they have recommended BankAmerica delay
its up to one-billion-dlr equity offering, which has yet to be
approved by the Securities and Exchange Commission.
    BankAmerica stock fell this week, along with other banking
issues, on the news that Brazil has suspended interest payments
on a large portion of its foreign debt.
    The stock traded around 12, down 1/8, this afternoon,
after falling to 11-1/2 earlier this week on the news.
    Banking analysts said that with the immediate threat of the
First Interstate Bancorp &lt;I> takeover bid gone, BankAmerica is
under no pressure to sell the securities into a market that
will be nervous on bank stocks in the near term.
    BankAmerica filed the offer on January 26. It was seen as
one of the major factors leading the First Interstat

# 3. Text Normalisation 

For most text analytics tasks, we will first need to transform the raw text to a suitable format for input to method such as a classifier. This process is called _text normalisation_ and is part of the _preprocessing_ stage. There are three common steps:

1. Sentence segmentation, which we have already been doing with regular expressions. 
2. Tokenisation, in which the sentences are split into a sequence of tokens, which include words, numbers and punctuation marks.
3. Word normalisation, in which different forms of a word are replaced by a root form.

We are now going to see how to perform these steps using the NLTK library.

## 3.1 Sentence Segmentation

Let's start by using NLTK to split a document into sentences. This should give better results than our regular expressions above:

In [115]:
import nltk

article = train_dataset[3]['text']
print(article)
sents = nltk.sent_tokenize(article)
print(len(sents))
for sent in sents[:5]:
    print("<SENTENCE>")
    print(sent)  # print the first five sentences of this document

BankAmerica Corp is not under
pressure to act quickly on its proposed equity offering and
would do well to delay it because of the stock's recent poor
performance, banking analysts said.
    Some analysts said they have recommended BankAmerica delay
its up to one-billion-dlr equity offering, which has yet to be
approved by the Securities and Exchange Commission.
    BankAmerica stock fell this week, along with other banking
issues, on the news that Brazil has suspended interest payments
on a large portion of its foreign debt.
    The stock traded around 12, down 1/8, this afternoon,
after falling to 11-1/2 earlier this week on the news.
    Banking analysts said that with the immediate threat of the
First Interstate Bancorp &lt;I> takeover bid gone, BankAmerica is
under no pressure to sell the securities into a market that
will be nervous on bank stocks in the near term.
    BankAmerica filed the offer on January 26. It was seen as
one of the major factors leading the First Interstate


TODO 4: Use the regular expression substitution code from section 2.2 to remove the new line characters from the sentences displayed above and print the results.

In [116]:
clean_sents = []

for sent in sents[:5]:
    
    ### COMPLETE YOUR CODE HERE
    sent = re.sub(r'([^\n]*)\n([^\n]*)',r'\1\2', sent)
    #######
    print("<SENTENCE>")
    print(sent)  #print the first five sentences of this document
    clean_sents.append(sent)  #save the cleaned sentences for later

<SENTENCE>
BankAmerica Corp is not underpressure to act quickly on its proposed equity offering andwould do well to delay it because of the stock's recent poorperformance, banking analysts said.
<SENTENCE>
Some analysts said they have recommended BankAmerica delayits up to one-billion-dlr equity offering, which has yet to beapproved by the Securities and Exchange Commission.
<SENTENCE>
BankAmerica stock fell this week, along with other bankingissues, on the news that Brazil has suspended interest paymentson a large portion of its foreign debt.
<SENTENCE>
The stock traded around 12, down 1/8, this afternoon,after falling to 11-1/2 earlier this week on the news.
<SENTENCE>
Banking analysts said that with the immediate threat of theFirst Interstate Bancorp &lt;I> takeover bid gone, BankAmerica isunder no pressure to sell the securities into a market thatwill be nervous on bank stocks in the near term.


## 3.2 Tokenisation

NLTK provides a similar function for tokenizing the text at the word level. You can find the documentation [here](https://www.nltk.org/api/nltk.tokenize.html). 

TODO 5: Use word_tokenize() to tokenize each of the sentences from the last cell.

In [117]:
tokenized_sents = []

for sent in clean_sents:
    ### WRITE YOUR OWN CODE HERE
    tokens = nltk.tokenize.word_tokenize(sent)
    #######
    
    print("<TOKENS>")
    print(tokens)
    
    tokenized_sents.append(tokens)

<TOKENS>
['BankAmerica', 'Corp', 'is', 'not', 'underpressure', 'to', 'act', 'quickly', 'on', 'its', 'proposed', 'equity', 'offering', 'andwould', 'do', 'well', 'to', 'delay', 'it', 'because', 'of', 'the', 'stock', "'s", 'recent', 'poorperformance', ',', 'banking', 'analysts', 'said', '.']
<TOKENS>
['Some', 'analysts', 'said', 'they', 'have', 'recommended', 'BankAmerica', 'delayits', 'up', 'to', 'one-billion-dlr', 'equity', 'offering', ',', 'which', 'has', 'yet', 'to', 'beapproved', 'by', 'the', 'Securities', 'and', 'Exchange', 'Commission', '.']
<TOKENS>
['BankAmerica', 'stock', 'fell', 'this', 'week', ',', 'along', 'with', 'other', 'bankingissues', ',', 'on', 'the', 'news', 'that', 'Brazil', 'has', 'suspended', 'interest', 'paymentson', 'a', 'large', 'portion', 'of', 'its', 'foreign', 'debt', '.']
<TOKENS>
['The', 'stock', 'traded', 'around', '12', ',', 'down', '1/8', ',', 'this', 'afternoon', ',', 'after', 'falling', 'to', '11-1/2', 'earlier', 'this', 'week', 'on', 'the', 'news', '.'

Run the code below to see how NLTK has handled the non-letter characters. 
* What does it do with most punctuation marks? Split the sentence based on the word
* When does it not split tokens based on punctuation? when it appears the words connected by "-" or the number format of the date.


In [118]:
for sent in tokenized_sents:
    for tok in sent:
        if re.search(r'[^a-zA-Z0-9]', tok):
            print(tok)

's
,
.
one-billion-dlr
,
.
,
,
.
,
1/8
,
,
11-1/2
.
&
;
>
,
.


## 3.3 Word Normalisation

Many words can appear in different forms, including: 
* Conjugated verbs
* Plural and singular nouns
* Common abbrevations and synonyms like "USA" and "US". 

Mapping all of these surface forms to a single root form reduces the size of the vocabulary that we have to deal with and can therefore improve the performance of text classifiers or topic models.

The two most widely used tools for this task in English are the Porter Stemmer and WordNet Lemmatizer. These tools apply a series of regular expression substitutions to tokenised text to convert words to a standard format. 
* The Porter stemmer is much faster but just removes word prefixes and endings, which leads to some errors. It is often used when real-time or high-volume text processing is needed.
* As well as applying regular expressions, lemmatizers look words up in a dictionary to find their root forms, so are more accurate but much slower. 

Let's start by applying the [Porter Stemmer class](https://www.nltk.org/_modules/nltk/stem/porter.html) to our tokenised text by calling the stem() method:

In [119]:
stemmer = nltk.PorterStemmer() 
stemmed_sents = []

for sent in tokenized_sents:
    stemmed_sent = [stemmer.stem(tok) for tok in sent]
    
    stemmed_sents.append(stemmed_sent)
    
    print("<STEMMED TOKENS>")
    print(stemmed_sent)

<STEMMED TOKENS>
['bankamerica', 'corp', 'is', 'not', 'underpressur', 'to', 'act', 'quickli', 'on', 'it', 'propos', 'equiti', 'offer', 'andwould', 'do', 'well', 'to', 'delay', 'it', 'becaus', 'of', 'the', 'stock', "'s", 'recent', 'poorperform', ',', 'bank', 'analyst', 'said', '.']
<STEMMED TOKENS>
['some', 'analyst', 'said', 'they', 'have', 'recommend', 'bankamerica', 'delayit', 'up', 'to', 'one-billion-dlr', 'equiti', 'offer', ',', 'which', 'ha', 'yet', 'to', 'beapprov', 'by', 'the', 'secur', 'and', 'exchang', 'commiss', '.']
<STEMMED TOKENS>
['bankamerica', 'stock', 'fell', 'thi', 'week', ',', 'along', 'with', 'other', 'bankingissu', ',', 'on', 'the', 'news', 'that', 'brazil', 'ha', 'suspend', 'interest', 'paymentson', 'a', 'larg', 'portion', 'of', 'it', 'foreign', 'debt', '.']
<STEMMED TOKENS>
['the', 'stock', 'trade', 'around', '12', ',', 'down', '1/8', ',', 'thi', 'afternoon', ',', 'after', 'fall', 'to', '11-1/2', 'earlier', 'thi', 'week', 'on', 'the', 'news', '.']
<STEMMED TOKENS

Now let's compare the stemming results to lemmatisation. For this task, NLTK provides the [class WordNetLemmatizer](https://www.nltk.org/_modules/nltk/stem/wordnet.html) with the method lemmatize(). This method takes an argument, `pos`, that determines whether the lemmatizer is applied to nouns, verbs, adjectives or adverbs.

TODO 6: Use the WordNetLemmatizer to lemmatize the nouns in the tokenized sentences. Set the `pos` argument to 'n'. 

TODO 7: Add a second call to lemmatize() to lemmatize the verbs in the sentences as well. Set the `pos` argument to 'v'. 

How do the results compare with the Porter stemmer? 

How have the verbs in the sentences changed?

In [131]:
lemmatizer = nltk.WordNetLemmatizer()
lemma_sents_n = []
lemma_sents_v = []
for sent in tokenized_sents:
    # WRITE YOUR OWN CODE HERE
    lemma_sent_n = []
    lemma_sent_v = []
    for word in sent:
        lemma_sent_n.append(lemmatizer.lemmatize(word, pos="n"))
        lemma_sent_v.append(lemmatizer.lemmatize(word, pos="v"))
    lemma_sents_n.append(lemma_sent_n)
    lemma_sents_v.append(lemma_sent_v)
    print("<LEMMATIZED TOKENS> for nouns")
    print(lemma_sent_n)
    print("<LEMMATIZED TOKENS> for verbs")
    print(lemma_sent_v)


<LEMMATIZED TOKENS> for nouns
['BankAmerica', 'Corp', 'is', 'not', 'underpressure', 'to', 'act', 'quickly', 'on', 'it', 'proposed', 'equity', 'offering', 'andwould', 'do', 'well', 'to', 'delay', 'it', 'because', 'of', 'the', 'stock', "'s", 'recent', 'poorperformance', ',', 'banking', 'analyst', 'said', '.']
<LEMMATIZED TOKENS> for verbs
['BankAmerica', 'Corp', 'be', 'not', 'underpressure', 'to', 'act', 'quickly', 'on', 'its', 'propose', 'equity', 'offer', 'andwould', 'do', 'well', 'to', 'delay', 'it', 'because', 'of', 'the', 'stock', "'s", 'recent', 'poorperformance', ',', 'bank', 'analysts', 'say', '.']
<LEMMATIZED TOKENS> for nouns
['Some', 'analyst', 'said', 'they', 'have', 'recommended', 'BankAmerica', 'delayits', 'up', 'to', 'one-billion-dlr', 'equity', 'offering', ',', 'which', 'ha', 'yet', 'to', 'beapproved', 'by', 'the', 'Securities', 'and', 'Exchange', 'Commission', '.']
<LEMMATIZED TOKENS> for verbs
['Some', 'analysts', 'say', 'they', 'have', 'recommend', 'BankAmerica', 'dela